<a href="https://colab.research.google.com/github/somendrew/LLMs/blob/main/Tool_Calling_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Steps
* Define Tool
* Describe Tool
* User Query
* Call LLM
* Check if tool is called
* Execute the function
* Send result back to model

In [38]:
import os

from google.colab import userdata
api_key = userdata.get('api_key')

In [39]:
from openai import OpenAI
import langchain

In [40]:
# Define and Describe tool
from langchain.tools import tool

@tool
def multiply(a: int,b: int ) -> int :
  "Multiply two integers and return the Result:"
  return a*b


In [41]:
#User Query

query = 'what is the multiplication of 3 and 6'
messages = [{'role':'user', 'content':query}]

In [42]:
messages

[{'role': 'user', 'content': 'what is the multiplication of 3 and 6'}]

In [49]:
from langchain_core.utils.function_calling import convert_to_openai_function

client = OpenAI(api_key=api_key)

# -----------------------------
# 2. Describe tool to the LLM
# -----------------------------
tools = [
    {
        "type": "function",
        "function": {
            "name": "multiply",
            "description": "Multiply two numbers together",
            "parameters": {
                "type": "object",
                "properties": {
                    "a": {
                        "type": "integer",
                        "description": "First number"
                    },
                    "b": {
                        "type": "integer",
                        "description": "Second number"
                    }
                },
                "required": ["a", "b"]
            }
        }
    }
]

response = client.chat.completions.create(
    model = 'gpt-4o',
    messages = messages,
    tools = tools,
    tool_choice ='auto'
)

In [44]:
import json
# Check if tool is called
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function
arguments = json.loads(tool_call.function.arguments)
tool_call

ChatCompletionMessageFunctionToolCall(id='call_RyXhLpP2FP1BbLlifJJ60mlY', function=Function(arguments='{"a":3,"b":6}', name='multiply'), type='function')

In [45]:
arguments

{'a': 3, 'b': 6}

In [46]:
response.choices[0].message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_RyXhLpP2FP1BbLlifJJ60mlY', function=Function(arguments='{"a":3,"b":6}', name='multiply'), type='function')])

In [57]:
# 1. Ensure we are accessing the tool call object correctly
tool_call = response.choices[0].message.tool_calls[0]
function_args = json.loads(tool_call.function.arguments)
function_name = tool_call.function.name

result = None

# 2. Execute the local function mapping
if function_name == "multiply":
    # Ensure 'multiply' is the tool object and 'func' is the callable
    result = multiply.func(a=function_args.get('a'), b=function_args.get('b'))
else:
    print(f"Warning: Unexpected function name: {function_name}")

# 3. Reconstruct messages (Crucial: tool_calls must be in the assistant message)
messages_for_final_call = [
    {"role": "user", "content": query},
    response.choices[0].message, # Keep the original assistant message object
    {
        "role": "tool",
        "tool_call_id": tool_call.id,
        "name": function_name,
        "content": str(result)
    }
]

# 4. Final API Call
final_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages_for_final_call
)

print(final_response.choices[0].message.content)

The multiplication of 3 and 6 is 18.
